In [4]:
print("Okay!!")

Okay!!


In [5]:
from langchain import PromptTemplate

In [6]:
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

c:\Users\pravi\anaconda3\envs\mchatbot\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [7]:
PINECONE_API_KEY = "bd5d2f2e-c096-4557-b8b3-53e92184bbe5"
from pinecone import Pinecone

In [8]:
def load_pdf(data):
    loader = DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [9]:
extracted_data = load_pdf("Data/")

In [10]:
def text_splitter(extracted_data):
    txt_split = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    text = txt_split.split_documents(extracted_data)
    return text

In [11]:
text_chunks = text_splitter(extracted_data)

In [12]:
len(text_chunks)

7020

In [13]:
def download_hfembeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [14]:
embeddings = download_hfembeddings()

In [15]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

In [16]:
from langchain.vectorstores import Chroma

In [17]:
vector_db = Chroma.from_documents(text_chunks, embeddings,persist_directory="db")

In [18]:
query = "What are allergies?"
docs = vector_db.similarity_search(query,k=3)

In [19]:
print(docs)

[Document(page_content="GALE ENCYCLOPEDIA OF MEDICINE 2 117Allergies\nAllergic rhinitis is commonly triggered by\nexposure to household dust, animal fur,or pollen. The foreign substance thattriggers an allergic reaction is calledan allergen.\nThe presence of an allergen causes the\nbody's lymphocytes to begin producingIgE antibodies. The lymphocytes of an allergy sufferer produce an unusuallylarge amount of IgE.\nIgE molecules attach to mast\ncells, which contain histamine.HistaminePollen grains\nLymphocyte\nFIRST EXPOSURE", metadata={'source': 'Data\\Medical_book.pdf', 'page': 130}), Document(page_content='the itchy, scratchy nose, eyes, and throat common inallergic rhinitis .\nThe particular allergens to which a person is sensi-', metadata={'source': 'Data\\Medical_book.pdf', 'page': 135}), Document(page_content='allergens are the following:\n• plant pollens\n• animal fur and dander\n• body parts from house mites (microscopic creatures\nfound in all houses)\n• house dust• mold spores

In [20]:
Prompt_template = """
                  Use the following pieces of information to answer the user's question. 
                  If you don't know the answer, just say that you don't know,don't try to make up an answer.

                  Context:{context}
                  Question:{question}

                  only return the help answer below nothing else.
                  Helpful answer:
"""

In [21]:
Prompt = PromptTemplate(template=Prompt_template,input_variables=["context", "question"])
chain_type_kwargs = {"prompt": Prompt}

In [22]:
llm = CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                    model_type="llama",
                    config={'max_new_tokens' : 512,
                            'temperature' : 0.8}
                    )

In [23]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vector_db.as_retriever(search_kwargs={'k':2}),
    chain_type_kwargs=chain_type_kwargs,    
    return_source_documents=True,
)

In [24]:
query = "What are the abnormal results of heart attack?"

In [25]:
result = qa(query)

In [26]:
result["result"]

"                  Heart attack can cause abnormal results such as:\n\n  Electrocardiogram (ECG): abnormal Q waves, ST segment changes, and T wave changes.\n  Blood tests: elevated levels of troponin, a protein released into the blood when heart muscle is damaged.\n  Chest X-ray: evidence of heart damage or enlargement.\n  Echocardiogram (echo): abnormal movement of the heart's walls and valves.\n  Magnetic resonance imaging (MRI): detailed images of the heart and its blood vessels.\n  Stress test: abnormal results indicate that the heart is not functioning properly during exercise.\n  Blood tests: elevated levels of creatine kinase-MB (CK-MB), a protein released into the blood when the heart muscle is damaged."